## Celda 1: Importaciones y Configuración Inicial del Logging


In [ ]:
import pandas as pd
import json
import os
import subprocess
import logging
from datetime import datetime

# --- Clase para Logger Estilizado (con caracteres ASCII simples para consola) ---
class StyledLogger:
    def __init__(self, logger_name='StyledLogger', log_file_path='agent.log', level=logging.INFO):
        self.logger = logging.getLogger(logger_name)
        self.logger.setLevel(level)
        self.logger.propagate = False 

        if self.logger.hasHandlers():
            self.logger.handlers.clear()

        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

        # File Handler con UTF-8
        try:
            fh = logging.FileHandler(log_file_path, encoding='utf-8')
            fh.setFormatter(formatter)
            self.logger.addHandler(fh)
        except Exception as e:
            print(f"Error al crear FileHandler para logs: {e}")

        # Stream Handler (consola del notebook)
        sh = logging.StreamHandler()
        sh.setFormatter(formatter)
        self.logger.addHandler(sh)

        # Estilos ASCII simplificados para mayor compatibilidad de consola
        self.HEADER_ART = """
        ********************************************************************************
        *                     G M A P S   S C R A P E R   A G E N T                    *
        *                         ( Avalian Project - MVP )                          *
        ********************************************************************************
        """
        self.SECTION_SEPARATOR = "=" * 80
        self.SUB_SECTION_SEPARATOR = "-" * 60
        self.SUCCESS_ART = "[ OK ]"
        self.ERROR_ART = "[FAIL]"
        self.WARNING_ART = "[WARN]"
        self.INFO_ART = "[INFO]"
        self.DEBUG_ART = "[DEBUG]"

    def _log(self, level, message, art=""):
        self.logger.log(level, f"{art} {message}".strip())

    def header(self):
        # Imprimir arte ASCII grande directamente a la consola del notebook
        # y un log formal al archivo de logs
        print(self.HEADER_ART)
        self.logger.info("Agent GOSOM MVP Initialized (inicio de sesión de logger).")

    def section(self, title):
        self.logger.info(f"\n{self.SECTION_SEPARATOR}\n{title.upper()}\n{self.SECTION_SEPARATOR}")

    def subsection(self, title):
        self.logger.info(f"\n{self.SUB_SECTION_SEPARATOR}\n{title}\n{self.SUB_SECTION_SEPARATOR}")

    def info(self, message):
        self._log(logging.INFO, message, self.INFO_ART)

    def success(self, message):
        self._log(logging.INFO, message, self.SUCCESS_ART)

    def warning(self, message):
        self._log(logging.WARNING, message, self.WARNING_ART)

    def error(self, message, exc_info=False):
        self._log(logging.ERROR, message, self.ERROR_ART)
        if exc_info: # Para capturar el traceback completo
            self.logger.exception("Detalles de la excepción:")

    def critical(self, message, exc_info=False):
        self._log(logging.CRITICAL, message, f"{self.ERROR_ART} [CRITICAL]")
        if exc_info:
            self.logger.exception("Detalles de la excepción crítica:")
            
    def debug(self, message):
        self._log(logging.DEBUG, message, self.DEBUG_ART)

# --- Configuración de Rutas y Logger ---
CONFIG_DIR = None
DATA_DIR = None
LOGS_DIR = None
config_params = {}
log_file_path_global = 'agent_gmaps_mvp_fallback.log'

try:
    # Asumimos que el notebook (.ipynb) está en la carpeta '0_AGENTE_GOSOM/notebooks/'
    # y el CWD es esa carpeta 'notebooks/'
    current_notebook_dir = os.getcwd()
    print(f"[DEBUG RUTA] Directorio actual del notebook (CWD): {current_notebook_dir}")

    # El directorio '0_AGENTE_GOSOM' es el padre de 'notebooks/'
    agent_gosom_dir = os.path.dirname(current_notebook_dir)
    print(f"[DEBUG RUTA] Directorio del agente GOSOM deducido: {agent_gosom_dir}")
    
    CONFIG_DIR = os.path.join(agent_gosom_dir, 'config')
    DATA_DIR = os.path.join(agent_gosom_dir, 'data') # Carpeta 'data' dentro de '0_AGENTE_GOSOM'
    LOGS_DIR = os.path.join(DATA_DIR, 'logs') # 'logs' dentro de 'data'

    config_file_path = os.path.join(CONFIG_DIR, 'parameters_default.json')
    print(f"[DEBUG RUTA] Intentando cargar config desde: {config_file_path}")

    with open(config_file_path, 'r', encoding='utf-8') as f:
        config_params = json.load(f)
    
    LOG_FILENAME = config_params.get('log_filename', 'agent_gmaps_mvp.log')
    
    os.makedirs(LOGS_DIR, exist_ok=True)
    log_file_path_global = os.path.join(LOGS_DIR, LOG_FILENAME)

    logger = StyledLogger(log_file_path=log_file_path_global, level=logging.INFO) # o logging.DEBUG
    logger.header()
    logger.success(f"Archivo de configuración '{config_file_path}' cargado correctamente.")

except FileNotFoundError:
    # Este print se verá incluso si el logger falla en instanciarse
    print(f"ERROR CRÍTICO [FileNotFoundError]: No se pudo encontrar el archivo de configuración en la ruta esperada: '{config_file_path}'. Por favor, verifica la estructura de carpetas y la ubicación del archivo 'parameters_default.json' en la carpeta 'config' relativa a '0_AGENTE_GOSOM'.")
    # Intentar instanciar logger con path de fallback para al menos loguear el error
    logger = StyledLogger(log_file_path=log_file_path_global) 
    logger.critical(f"Archivo de parámetros '{config_file_path}' NO ENCONTRADO. Usando defaults para logger y parámetros.", exc_info=False) # No queremos exc_info si el problema es FileNotFoundError aquí
except Exception as e:
    print(f"ERROR CRÍTICO al cargar configuración o inicializar logger: {e}")
    logger = StyledLogger(log_file_path=log_file_path_global)
    logger.critical(f"Error al cargar configuración o inicializar logger: {e}", exc_info=True)


# --- Carga de Parámetros Globales (Continuación) ---
logger.section("Cargando Parámetros Globales del Agente")

LANGUAGE = config_params.get('language', 'es')
logger.info(f"Lenguaje para scraping: {LANGUAGE}")

DEFAULT_DEPTH = config_params.get('default_depth', 2)
logger.info(f"Profundidad de búsqueda por defecto: {DEFAULT_DEPTH}")

RESULTS_FILENAME_PREFIX = config_params.get('results_filename_prefix', 'gmaps_data_')
logger.info(f"Prefijo para archivos de resultados: {RESULTS_FILENAME_PREFIX}")

# Nombres de las subcarpetas de datos (del JSON)
RAW_CSV_FOLDER_NAME = config_params.get('output_csv_folder_raw_name', 'raw')
PROCESSED_CSV_FOLDER_NAME = config_params.get('output_csv_folder_processed_name', 'processed')

# Rutas absolutas a las carpetas de datos (si DATA_DIR se definió correctamente)
if DATA_DIR:
    RAW_CSV_FOLDER = os.path.join(DATA_DIR, RAW_CSV_FOLDER_NAME)
    PROCESSED_CSV_FOLDER = os.path.join(DATA_DIR, PROCESSED_CSV_FOLDER_NAME)

    logger.info(f"Carpeta para CSVs crudos: {RAW_CSV_FOLDER}")
    logger.info(f"Carpeta para CSVs procesados: {PROCESSED_CSV_FOLDER}")

    # Crear carpetas de datos si no existen
    try:
        os.makedirs(RAW_CSV_FOLDER, exist_ok=True)
        os.makedirs(PROCESSED_CSV_FOLDER, exist_ok=True)
        logger.success("Carpetas de datos RAW y PROCESSED verificadas/creadas.")
    except Exception as e:
        logger.error(f"No se pudieron crear las carpetas de datos '{RAW_CSV_FOLDER}' o '{PROCESSED_CSV_FOLDER}': {e}", exc_info=True)
else:
    logger.error("DATA_DIR no está definido. No se pueden establecer rutas para CSVs crudos/procesados.")
    RAW_CSV_FOLDER = None # Asegurar que estén definidos aunque sea como None
    PROCESSED_CSV_FOLDER = None


GMAPS_COORDINATES = config_params.get('gmaps_coordinates', {})
if GMAPS_COORDINATES:
    logger.info(f"Coordenadas cargadas para {len(GMAPS_COORDINATES)} ciudades.")
else:
    logger.warning("No se cargaron coordenadas de ciudades desde la configuración.")

logger.info("Celda 1 completada: Logger y configuración base listos.")

2025-05-28 14:38:17 - INFO - Agent GOSOM MVP Initialized (inicio de sesión de logger).
2025-05-28 14:38:17 - INFO - [ OK ] Archivo de configuración 'c:\Users\El Bauto\Documents\MEGA\Generative_ai\etl-gmaps\0_AGENTE_GOSOM\config\parameters_default.json' cargado correctamente.
2025-05-28 14:38:17 - INFO - 
CARGANDO PARÁMETROS GLOBALES DEL AGENTE
2025-05-28 14:38:17 - INFO - [INFO] Lenguaje para scraping: es
2025-05-28 14:38:17 - INFO - [INFO] Profundidad de búsqueda por defecto: 2
2025-05-28 14:38:17 - INFO - [INFO] Prefijo para archivos de resultados: gmaps_data_
2025-05-28 14:38:17 - INFO - [INFO] Carpeta para CSVs crudos: c:\Users\El Bauto\Documents\MEGA\Generative_ai\etl-gmaps\0_AGENTE_GOSOM\data\raw
2025-05-28 14:38:17 - INFO - [INFO] Carpeta para CSVs procesados: c:\Users\El Bauto\Documents\MEGA\Generative_ai\etl-gmaps\0_AGENTE_GOSOM\data\processed
2025-05-28 14:38:17 - INFO - [ OK ] Carpetas de datos RAW y PROCESSED verificadas/creadas.
2025-05-28 14:38:17 - INFO - [INFO] Coordena

[DEBUG RUTA] Directorio del agente GOSOM asumido: c:\Users\El Bauto\Documents\MEGA\Generative_ai\etl-gmaps\0_AGENTE_GOSOM
[DEBUG RUTA] Intentando cargar config desde: c:\Users\El Bauto\Documents\MEGA\Generative_ai\etl-gmaps\0_AGENTE_GOSOM\config\parameters_default.json

        ********************************************************************************
        *                     G M A P S   S C R A P E R   A G E N T                    *
        *                         ( Avalian Project - MVP )                          *
        ********************************************************************************
        


## Celda 2 (Revisada): Funciones para Leer Archivos de Keywords


In [2]:
# Celda 2: Funciones para Leer Archivos de Keywords

logger.section("Definiendo Funciones de Utilidad")
logger.subsection("Función para Cargar Keywords")

def load_keywords_from_csv(city_name_key):
    """
    Carga keywords desde un archivo CSV específico para una clave de ciudad.
    La clave de ciudad se usa para encontrar el archivo en 'config/keywords_<city_name_key>.csv'
    """
    if CONFIG_DIR is None:
        logger.error("Directorio de configuración (CONFIG_DIR) no está definido. No se pueden cargar keywords.")
        return []

    filepath = os.path.join(CONFIG_DIR, f'keywords_{city_name_key.lower()}.csv')
    
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            keywords = [line.strip() for line in f if line.strip()] # Filtra líneas vacías y quita espacios
        
        if keywords:
            logger.success(f"Keywords cargadas para '{city_name_key}' desde '{filepath}': {len(keywords)} keywords.")
        else:
            logger.warning(f"No se encontraron keywords válidas (o el archivo está vacío) en '{filepath}' para '{city_name_key}'.")
        return keywords
    except FileNotFoundError:
        logger.error(f"Archivo de keywords no encontrado para '{city_name_key}' en '{filepath}'.")
        return []
    except Exception as e:
        logger.error(f"Error al cargar keywords para '{city_name_key}' desde '{filepath}': {e}", exc_info=True)
        return []

# --- Pruebas de la función load_keywords_from_csv (Opcional) ---
logger.subsection("Probando Carga de Keywords")

# Prueba con una ciudad que debería tener archivo de keywords
test_city_existing = 'neuquen' # Asegúrate de tener '0_AGENTE_GOSOM/config/keywords_neuquen.csv'
logger.info(f"Intentando cargar keywords para la ciudad de prueba: '{test_city_existing}'")
keywords_test_existing = load_keywords_from_csv(test_city_existing)

if keywords_test_existing:
    logger.info(f"Ejemplo de keywords para {test_city_existing.capitalize()} (primeras 3 si hay): {keywords_test_existing[:3]}")
else:
    # El warning o error ya fue logueado por la función load_keywords_from_csv
    pass

# Prueba con una ciudad que NO debería tener archivo de keywords
test_city_non_existing = 'ciudad_totalmente_inventada'
logger.info(f"Intentando cargar keywords para una ciudad de prueba que no existe: '{test_city_non_existing}'")
keywords_test_non_existing = load_keywords_from_csv(test_city_non_existing)
# (Se espera un error en el log si el archivo no existe, y la función devuelve [])

logger.info("Celda 2 completada: Función de carga de keywords definida y probada.")

2025-05-28 14:38:33 - INFO - 
DEFINIENDO FUNCIONES DE UTILIDAD
2025-05-28 14:38:33 - INFO - 
------------------------------------------------------------
Función para Cargar Keywords
------------------------------------------------------------
2025-05-28 14:38:33 - INFO - 
------------------------------------------------------------
Probando Carga de Keywords
------------------------------------------------------------
2025-05-28 14:38:33 - INFO - [INFO] Intentando cargar keywords para la ciudad de prueba: 'neuquen'
2025-05-28 14:38:33 - INFO - [ OK ] Keywords cargadas para 'neuquen' desde 'c:\Users\El Bauto\Documents\MEGA\Generative_ai\etl-gmaps\0_AGENTE_GOSOM\config\keywords_neuquen.csv': 4 keywords.
2025-05-28 14:38:33 - INFO - [INFO] Ejemplo de keywords para Neuquen (primeras 3 si hay): ['Contadores en Neuquen', 'Empresas de servicios en Neuquen', 'Consultorios medicos en Neuquen']
2025-05-28 14:38:33 - INFO - [INFO] Intentando cargar keywords para una ciudad de prueba que no exist

## Celda 3 (Completa y Arreglada): Función para Ejecutar el Scraper GOSOM


In [ ]:
# Celda 3: Función para Ejecutar el Scraper GOSOM (usando Docker)

logger.section("Definiendo Función para Ejecutar GOSOM Scraper vía Docker")

def run_gmaps_scraper_docker(keywords_list, city_name_key, depth_override=None):
    """
    Ejecuta el scraper GOSOM usando Docker para una lista de keywords y una clave de ciudad.
    La clave de ciudad se usa para obtener coordenadas y nombrar archivos.
    Guarda los resultados en un archivo CSV en la carpeta RAW_CSV_FOLDER.
    """
    logger.subsection(f"Iniciando scraping para ciudad: '{city_name_key.capitalize()}'")

    if not keywords_list:
        logger.warning(f"No hay keywords para scrapear para '{city_name_key}'. Omitiendo.")
        return None

    if not CONFIG_DIR or not RAW_CSV_FOLDER:
        logger.error("CONFIG_DIR o RAW_CSV_FOLDER no están definidos. No se puede ejecutar el scraper.")
        return None

    # Obtener coordenadas para la ciudad
    city_info = GMAPS_COORDINATES.get(city_name_key.lower())
    if not city_info:
        logger.error(f"No se encontraron coordenadas/info para '{city_name_key}' en la configuración (GMAPS_COORDINATES).")
        return None
    
    lat = city_info.get('latitude')
    lon = city_info.get('longitude')
    # radius = city_info.get('radius', 10000) # Radius es principalmente para fast-mode
    # zoom = city_info.get('zoom', 14)      # Zoom es principalmente para fast-mode
    
    if lat is None or lon is None:
        logger.error(f"Latitud o longitud faltantes para '{city_name_key}'.")
        return None

    current_depth = depth_override if depth_override is not None else DEFAULT_DEPTH
    logger.info(f"Usando profundidad de búsqueda: {current_depth}")

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    raw_output_filename = f"{RESULTS_FILENAME_PREFIX}{city_name_key.lower()}_{timestamp}.csv"
    raw_output_filepath_host = os.path.join(RAW_CSV_FOLDER, raw_output_filename)
    raw_output_filepath_container = f"/app/data/raw/{raw_output_filename}" # Ruta DENTRO del contenedor

    # Crear un archivo temporal para las queries en la carpeta config del HOST
    temp_queries_filename = f"temp_queries_{city_name_key.lower()}_{timestamp}.txt"
    temp_queries_filepath_host = os.path.join(CONFIG_DIR, temp_queries_filename)
    temp_queries_filepath_container = f"/app/config/{temp_queries_filename}" # Ruta DENTRO del contenedor

    try:
        logger.info(f"Creando archivo temporal de queries en: {temp_queries_filepath_host}")
        with open(temp_queries_filepath_host, 'w', encoding='utf-8') as f:
            for kw in keywords_list:
                f.write(f"{kw}\n")
        logger.success(f"Archivo temporal de queries creado con {len(keywords_list)} keywords.")

        # GOSOM requiere que el archivo de resultados exista ANTES de ejecutarlo.
        logger.info(f"Creando archivo de salida CSV vacío en: {raw_output_filepath_host}")
        with open(raw_output_filepath_host, 'w', encoding='utf-8') as f:
            pass # Simplemente crea el archivo vacío
        logger.success("Archivo de salida CSV vacío creado.")
        
        # Construcción del comando Docker
        # Montamos la carpeta 'config' del host (que contiene temp_queries) a '/app/config' en el contenedor
        # Montamos la carpeta 'data/raw' del host a '/app/data/raw' en el contenedor
        docker_command = [
            "docker", "run",
            "--rm", 
            "-v", f"{CONFIG_DIR}:/app/config:ro",  # :ro para read-only si el contenedor no necesita escribir en config
            "-v", f"{RAW_CSV_FOLDER}:/app/data/raw",
            "gosom/google-maps-scraper",
            "-lang", LANGUAGE,
            "-depth", str(current_depth),
            "-input", temp_queries_filepath_container,
            "-results", raw_output_filepath_container,
            "-exit-on-inactivity", "3m",
            "-geo", f"{lat},{lon}"
            # Si quieres activar emails (tarda mucho más):
            # "-email"
        ]

        logger.info(f"Ejecutando comando Docker: {' '.join(docker_command)}")
        
        # Ejecutar el comando
        process = subprocess.run(docker_command, capture_output=True, text=True, encoding='utf-8', check=False)

        if process.returncode == 0:
            logger.success(f"Scraping completado exitosamente para '{city_name_key}'.")
            logger.info(f"Resultados guardados en: '{raw_output_filepath_host}'")
            if process.stdout:
                logger.debug(f"Salida estándar del scraper (stdout):\n{process.stdout[:500]}...") # Muestra solo una parte
            if process.stderr:
                logger.warning(f"Salida de error estándar del scraper (stderr), si la hubo:\n{process.stderr[:500]}...")
            return raw_output_filepath_host
        else:
            logger.error(f"Error durante el scraping para '{city_name_key}'. Código de retorno: {process.returncode}")
            if process.stdout:
                logger.error(f"Salida estándar del scraper (stdout) en error:\n{process.stdout}")
            if process.stderr:
                logger.error(f"Salida de error estándar del scraper (stderr) en error:\n{process.stderr}")
            return None

    except FileNotFoundError as fnf_error: # Específico para el comando docker si no se encuentra
        logger.critical(f"Comando 'docker' no encontrado. Asegúrate de que Docker esté instalado y en el PATH del sistema.", exc_info=True)
        return None
    except Exception as e:
        logger.critical(f"Excepción inesperada al ejecutar el scraper para '{city_name_key}': {e}", exc_info=True)
        return None
    finally:
        # Eliminar el archivo temporal de queries
        if os.path.exists(temp_queries_filepath_host):
            try:
                os.remove(temp_queries_filepath_host)
                logger.info(f"Archivo temporal de queries eliminado: {temp_queries_filepath_host}")
            except Exception as e_remove:
                logger.warning(f"No se pudo eliminar el archivo temporal de queries '{temp_queries_filepath_host}': {e_remove}")

# --- Prueba de la función run_gmaps_scraper_docker ---
# Descomenta CON CUIDADO para probar una ejecución real. Asegúrate de que Docker Desktop esté corriendo.
# Y que tengas 'keywords_neuquen.csv' en '0_AGENTE_GOSOM/config/'
# Y que 'neuquen' esté definido en GMAPS_COORDINATES en 'parameters_default.json'


'''
 logger.section("Probando Ejecución del Scraper GOSOM para Neuquén")
 test_city_for_scraping = 'neuquen'
 keywords_for_scraping_test = load_keywords_from_csv(test_city_for_scraping)

 if keywords_for_scraping_test and GMAPS_COORDINATES.get(test_city_for_scraping.lower()):
     logger.info(f"Iniciando prueba de scraping real para: {test_city_for_scraping.capitalize()}")
     # Usar una profundidad muy baja para la prueba
     results_file = run_gmaps_scraper_docker(keywords_for_scraping_test, test_city_for_scraping, depth_override=1) 
     if results_file:
         logger.success(f"PRUEBA DE SCRAPING FINALIZADA. Archivo generado: {results_file}")
         # Podrías añadir aquí una lectura rápida del CSV para ver si tiene datos
         try:
             df_test_results = pd.read_csv(results_file)
             logger.info(f"El archivo de resultados contiene {len(df_test_results)} filas.")
             if not df_test_results.empty:
                 display(df_test_results.head(2))
         except Exception as e_read:
             logger.error(f"No se pudo leer el archivo de resultados de prueba: {e_read}")
     else:
         logger.error(f"PRUEBA DE SCRAPING FALLÓ para {test_city_for_scraping.capitalize()}. Revisa los logs.")
 else:
     logger.warning(f"No se ejecutará la prueba de scraping. Faltan keywords o coordenadas para '{test_city_for_scraping}'.")
'''


logger.info("Celda 3 completada: Función para ejecutar GOSOM vía Docker definida.")

## Celda 4: Orquestación del Proceso de Scraping para Ciudades Definidas


Esta celda utilizará las funciones de las celdas anteriores para iterar sobre una lista de ciudades (o todas las que tengan keywords y coordenadas), ejecutar el scraper para cada una, y recolectar las rutas a los archivos CSV crudos generados.

In [ ]:
# Celda 4: Orquestación del Proceso de Scraping

logger.section("Orquestando Tareas de Scraping")

# Lista de ciudades a procesar. Podrías obtenerlas de las claves en GMAPS_COORDINATES
# o tener una lista explícita.
# Por ahora, procesaremos solo las ciudades para las que tengamos archivos de keywords y coordenadas.

cities_to_process = []
if GMAPS_COORDINATES and CONFIG_DIR:
    for city_key in GMAPS_COORDINATES.keys():
        # Verificar si existe un archivo de keywords para esta ciudad
        keywords_file_path = os.path.join(CONFIG_DIR, f'keywords_{city_key.lower()}.csv')
        if os.path.exists(keywords_file_path):
            cities_to_process.append(city_key)
            logger.info(f"Ciudad '{city_key.capitalize()}' añadida a la lista de procesamiento (tiene coordenadas y archivo de keywords).")
        else:
            logger.warning(f"No se encontró archivo de keywords para '{city_key}' en '{keywords_file_path}'. Se omitirá esta ciudad.")
else:
    logger.error("No hay coordenadas (GMAPS_COORDINATES) o directorio de configuración (CONFIG_DIR) definido. No se pueden determinar ciudades a procesar.")


raw_csv_files_generated = {} # Diccionario para guardar: {'ciudad': 'ruta_al_csv_crudo'}

if not cities_to_process:
    logger.warning("No hay ciudades configuradas para procesar. Revisa GMAPS_COORDINATES y los archivos 'keywords_ciudad.csv'.")
else:
    logger.info(f"Ciudades a procesar en esta ejecución: {', '.join([c.capitalize() for c in cities_to_process])}")
    
    for city_key in cities_to_process:
        logger.subsection(f"Procesando Ciudad: {city_key.capitalize()}")
        
        keywords = load_keywords_from_csv(city_key)
        if not keywords:
            logger.warning(f"No se cargaron keywords para {city_key.capitalize()}, se omite el scraping para esta ciudad.")
            continue

        # Aquí puedes decidir la profundidad. Podrías tenerla en GMAPS_COORDINATES por ciudad,
        # o usar un valor diferente para pruebas.
        # depth_for_city = GMAPS_COORDINATES[city_key].get('depth', DEFAULT_DEPTH) 
        depth_for_city = 1 # Para el MVP y pruebas rápidas, usar una profundidad baja.
        
        logger.info(f"Intentando scraping para {city_key.capitalize()} con {len(keywords)} keywords y profundidad {depth_for_city}.")
        
        # --- ESTA LÍNEA EJECUTARÁ DOCKER ---
        # (En la prueba real, esto puede tardar varios minutos por ciudad)
        # raw_file_path = run_gmaps_scraper_docker(keywords, city_key, depth_override=depth_for_city)
        # --- FIN DE LÍNEA QUE EJECUTA DOCKER ---

        # --- PARA DESARROLLO SIN EJECUTAR DOCKER CADA VEZ ---
        # Simula que el scraper se ejecutó y generó un archivo.
        # Asegúrate de tener un archivo de ejemplo en data/raw/ para que esto funcione sin Docker.
        # Por ejemplo: data/raw/gmaps_data_neuquen_dummy.csv
        logger.warning("MODO SIMULACIÓN: `run_gmaps_scraper_docker` está COMENTADO. Usando archivo dummy si existe.")
        dummy_file_name = f"{RESULTS_FILENAME_PREFIX}{city_key.lower()}_dummy.csv"
        dummy_file_path = os.path.join(RAW_CSV_FOLDER, dummy_file_name)
        if os.path.exists(dummy_file_path):
            raw_file_path = dummy_file_path
            logger.info(f"MODO SIMULACIÓN: Usando archivo dummy existente: {raw_file_path}")
        else:
            raw_file_path = None
            logger.warning(f"MODO SIMULACIÓN: Archivo dummy NO encontrado en {dummy_file_path}. No habrá datos para esta ciudad en simulación.")
        # --- FIN DE SECCIÓN DE SIMULACIÓN ---

        if raw_file_path and os.path.exists(raw_file_path):
            raw_csv_files_generated[city_key] = raw_file_path
            logger.success(f"Scraping para {city_key.capitalize()} simulado/completado. Archivo: {raw_file_path}")
        else:
            logger.error(f"Scraping para {city_key.capitalize()} falló o no generó archivo (o archivo dummy no encontrado).")

if raw_csv_files_generated:
    logger.success(f"Proceso de scraping (o simulación) finalizado. Archivos crudos generados:")
    for city, path in raw_csv_files_generated.items():
        logger.info(f"  - {city.capitalize()}: {path}")
else:
    logger.warning("No se generaron archivos CSV crudos en esta ejecución.")

logger.info("Celda 4 completada: Orquestación de scraping (o simulación) definida.")

## Celda 5: Carga y Transformación de Datos


In [ ]:
# Celda 5: Carga y Transformación de Datos

logger.section("Cargando y Transformando Datos Extraídos")

# Nombres de las columnas esperadas del CSV de GOSOM
# ¡IMPORTANTE! Asegúrate de que el orden y los nombres coincidan con la salida real de GOSOM.
# Esta lista debe ser igual a la que usaste en tu EDA.
gmaps_column_names = [
    'input_id', 'link', 'title', 'category', 'address', 'open_hours', 'popular_times',
    'website', 'phone', 'plus_code', 'review_count', 'review_rating', 'reviews_per_rating',
    'latitude', 'longitude', 'cid', 'status', 'descriptions', 'reviews_link', 'thumbnail',
    'timezone', 'price_range', 'data_id', 'images', 'reservations', 'order_online',
    'menu', 'owner', 'complete_address', 'about', 'user_reviews', 'emails'
    # Añadir 'user_reviews_extended' si se usa el flag --extra-reviews
]

# Lista para almacenar todos los DataFrames procesados de cada ciudad
all_processed_data = []

# Función de transformación (puedes expandirla mucho más)
def transform_gmaps_data(df_raw, city_key_origin):
    logger.subsection(f"Transformando datos para: {city_key_origin.capitalize()}")
    df = df_raw.copy()

    # 1. Convertir tipos de datos
    logger.info("Convirtiendo tipos de datos numéricos...")
    for col in ['review_count', 'review_rating', 'latitude', 'longitude']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        else:
            logger.warning(f"Columna '{col}' no encontrada para conversión numérica.")
            df[col] = pd.NA # Añadirla como NA si no existe para evitar errores posteriores

    # 2. Parsear 'complete_address' para extraer componentes
    # (Reutilizando la función del EDA, asegúrate de que esté definida si no la copiaste aquí)
    def extract_address_components(json_str_or_data):
        try:
            if pd.isna(json_str_or_data): return pd.Series([None, None, None, None, None], index=['parsed_street', 'parsed_city_comp', 'parsed_postal_code', 'parsed_state', 'parsed_country'])
            
            data = json.loads(str(json_str_or_data)) if isinstance(json_str_or_data, str) else json_str_or_data
            
            if isinstance(data, dict):
                return pd.Series([
                    data.get('street'), data.get('city'), data.get('postal_code'), 
                    data.get('state'), data.get('country')
                ], index=['parsed_street', 'parsed_city_comp', 'parsed_postal_code', 'parsed_state', 'parsed_country'])
            # Añadir manejo si 'data' es una lista, si es necesario basado en tu EDA
            return pd.Series([None, None, None, None, None], index=['parsed_street', 'parsed_city_comp', 'parsed_postal_code', 'parsed_state', 'parsed_country'])
        except: # Captura amplia para evitar que falle la transformación
            return pd.Series([None, None, None, None, None], index=['parsed_street', 'parsed_city_comp', 'parsed_postal_code', 'parsed_state', 'parsed_country'])

    if 'complete_address' in df.columns:
        logger.info("Parseando 'complete_address'...")
        address_components = df['complete_address'].apply(extract_address_components)
        df = pd.concat([df, address_components], axis=1)
        logger.success("Componentes de dirección parseados y añadidos.")
    else:
        logger.warning("Columna 'complete_address' no encontrada. No se parsearán componentes de dirección.")


    # 3. Añadir columna de origen (la ciudad de la búsqueda)
    df['search_origin_city'] = city_key_origin.capitalize()
    logger.info(f"Añadida columna 'search_origin_city' con valor '{city_key_origin.capitalize()}'.")

    # 4. Selección de columnas relevantes para Avalian (ejemplo)
    # Ajusta esta lista según las necesidades reales de tu BD y CRM
    relevant_columns = [
        'title', 'category', 'address', 'parsed_street', 'parsed_city_comp', 'parsed_postal_code', 'parsed_state', 
        'website', 'phone', 'review_count', 'review_rating', 'latitude', 'longitude', 
        'link', 'search_origin_city' # 'link' es el enlace a Gmaps, 'search_origin_city' es la que añadimos
        # Considera también: 'cid', 'status', 'open_hours'
    ]
    # Filtrar para mantener solo columnas que realmente existen en el df después de transformaciones
    final_columns = [col for col in relevant_columns if col in df.columns]
    if len(final_columns) < len(relevant_columns):
        missing_for_selection = set(relevant_columns) - set(final_columns)
        logger.warning(f"Algunas columnas relevantes para selección no existen en el DataFrame: {missing_for_selection}")
    
    df_processed = df[final_columns].copy() # Usar .copy() para evitar SettingWithCopyWarning
    logger.success(f"Seleccionadas {len(df_processed.columns)} columnas relevantes.")
    
    # 5. Limpieza adicional (ejemplos)
    if 'phone' in df_processed.columns:
        df_processed.loc[:, 'phone'] = df_processed['phone'].str.replace(r'[^\d\+\s]', '', regex=True).str.strip() # Limpiar caracteres no deseados del teléfono
    
    logger.info(f"Transformación completada para {city_key_origin.capitalize()}. {len(df_processed)} registros procesados.")
    return df_processed


if not raw_csv_files_generated:
    logger.warning("No hay archivos CSV crudos para procesar (variable 'raw_csv_files_generated' está vacía).")
else:
    for city_key, raw_file_path in raw_csv_files_generated.items():
        logger.subsection(f"Procesando archivo crudo para: {city_key.capitalize()} ({raw_file_path})")
        try:
            # Leer el CSV crudo. GOSOM no escribe encabezado.
            df_raw_city = pd.read_csv(raw_file_path, header=None, names=gmaps_column_names, on_bad_lines='skip')
            logger.info(f"Leído archivo CSV crudo para {city_key.capitalize()}. {len(df_raw_city)} filas encontradas.")
            
            if not df_raw_city.empty:
                df_processed_city = transform_gmaps_data(df_raw_city, city_key)
                all_processed_data.append(df_processed_city)
                logger.success(f"Datos de {city_key.capitalize()} transformados y añadidos a la lista general.")
            else:
                logger.warning(f"El archivo CSV para {city_key.capitalize()} estaba vacío. No se procesaron datos.")
        except pd.errors.EmptyDataError:
            logger.warning(f"El archivo CSV para {city_key.capitalize()} en '{raw_file_path}' está vacío o es inválido.")
        except Exception as e:
            logger.error(f"Error al procesar el archivo para {city_key.capitalize()} ({raw_file_path}): {e}", exc_info=True)

if all_processed_data:
    # Combinar todos los DataFrames procesados en uno solo
    final_df_all_cities = pd.concat(all_processed_data, ignore_index=True)
    logger.success(f"Todos los datos procesados combinados. Total de {len(final_df_all_cities)} prospectos.")
    logger.info("Muestra de los primeros 5 prospectos combinados:")
    display(final_df_all_cities.head())
else:
    logger.warning("No se procesaron datos de ninguna ciudad. El DataFrame final está vacío.")
    final_df_all_cities = pd.DataFrame() # Crear un DF vacío para que el resto del notebook no falle

logger.info("Celda 5 completada: Carga y transformación de datos definida.")

In [ ]:
# Ejemplo de deduplicación (añadir después de concatenar en Celda 5)
if not final_df_all_cities.empty:
    logger.subsection("Aplicando Deduplicación de Prospectos")
    # Deduplicar basado en el link de Google Maps, manteniendo la primera aparición
    # Podrías necesitar una lógica más sofisticada si los links no son siempre perfectos o si quieres fusionar info
    initial_count = len(final_df_all_cities)
    final_df_all_cities.drop_duplicates(subset=['link'], keep='first', inplace=True)
    deduplicated_count = initial_count - len(final_df_all_cities)
    if deduplicated_count > 0:
        logger.success(f"Se eliminaron {deduplicated_count} prospectos duplicados basados en la columna 'link'.")
    else:
        logger.info("No se encontraron duplicados basados en la columna 'link' o ya estaban limpios.")
    logger.info(f"Total de prospectos únicos después de deduplicación: {len(final_df_all_cities)}")


## Celda 6: Guardado de Datos Procesados (a un único CSV por ahora)


In [ ]:
# Celda 6: Guardado de Datos Procesados

logger.section("Guardando Datos Procesados y Limpios")

if not final_df_all_cities.empty:
    timestamp_save = datetime.now().strftime("%Y%m%d_%H%M%S")
    processed_filename = f"gmaps_prospectos_consolidados_{timestamp_save}.csv"
    
    if PROCESSED_CSV_FOLDER: # Verificar que la carpeta de destino esté definida
        processed_filepath = os.path.join(PROCESSED_CSV_FOLDER, processed_filename)
        try:
            final_df_all_cities.to_csv(processed_filepath, index=False, encoding='utf-8-sig') # utf-8-sig para mejor compatibilidad con Excel
            logger.success(f"Datos procesados consolidados guardados en: {processed_filepath}")
            logger.info(f"Total de {len(final_df_all_cities)} prospectos guardados.")
        except Exception as e:
            logger.error(f"Error al guardar el archivo CSV procesado en '{processed_filepath}': {e}", exc_info=True)
    else:
        logger.error("PROCESSED_CSV_FOLDER no está definido. No se puede guardar el archivo procesado.")
elif raw_csv_files_generated and not all_processed_data : # Hubo archivos crudos pero no se procesó nada
    logger.warning("Se generaron archivos crudos pero no se pudieron procesar o resultaron vacíos. No se guardará archivo consolidado.")
else: # No hubo archivos crudos para empezar
    logger.warning("No hay datos procesados para guardar (DataFrame final está vacío).")

logger.info("Celda 6 completada: Proceso de guardado de datos procesados definido.")